# Import libraries and data

In [1]:
import numpy as np
import pandas as pd

In [2]:
url = 'https://drive.google.com/file/d/1-lhxHcH5YGPDg28rJuivfO8q27wvfiEl/view'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
raw = pd.read_csv(path)

Visits before 2023 won't be considered

In [3]:
raw['date'] = pd.to_datetime(raw['date'])
raw_2023 = raw[raw.date.dt.year==2023]

# Explode topics

In [4]:
raw_2023 = raw_2023.assign(
    topic_cnt = raw_2023.topic.apply(lambda s:len(s.split(';'))),
    topic = raw_2023.topic.str.split(';')
)


In [5]:
topic_expl = raw_2023.explode('topic')
topic_expl.reset_index(inplace=True)
topic_expl.drop('index', axis=1, inplace=True)

# Sum up visits

In [6]:
df = (topic_expl.groupby(['country', 'scv_id', 'source_system', 'topic']).count().iloc[:,0]).reset_index()
df.rename(columns={df.columns[-1]:'events'}, inplace=True)

In [7]:
df = (df.pivot(columns=['source_system'], index=['country', 'scv_id', 'topic'])
      .reset_index()
      .fillna(0)   
     )

In [8]:
df.columns = ['_'.join(c) for c in df.columns.to_flat_index()]
cut_dash = lambda c: c[:len(c)-1] if c[-1]=='_' else c
df.columns = [cut_dash(c) for c in df.columns]
df['events'] = df.events_PX + df.events_activecampaign + df.events_catalogue

Knowing that the distribution of visits count is extremally right skewed, setting up treshhold above which all affinity scores will be set to maximum

In [9]:
def outlier_tresh(x):
  multiplier = 4 #this may be edited
  iqr = x.quantile(.75)-x.quantile(.25)
  med = x.median()
  return med+iqr*4

In [10]:
tresh = outlier_tresh(df.events)

# Build up schema
Table with all scv_ids and all combination for source_system and topic

In [11]:
topics = (df.groupby(['country', 'topic']).count().iloc[:,0]).reset_index()
topics.drop(topics.columns[-1], axis=1, inplace=True)

In [12]:
ids = df['scv_id'].unique()

Merging list of scv_ids, combination source_system and topic, and data

In [20]:
result = (pd.DataFrame({'scv_id':ids}).merge(topics, how='cross')
    .merge(df, on=['scv_id', 'topic', 'country'], how='left')
    .fillna(0)
)


Shrink numbers

In [21]:
result['events_PX'] = result['events_PX'].astype('int16')
result['events_activecampaign'] = result['events_activecampaign'].astype('int16')
result['events_catalogue'] = result['events_catalogue'].astype('int16')
result['events'] = result['events'].astype('int16')
result['scv_id'] = result['scv_id'].astype('int32')

result['country'] = result['country'].astype('category')
result['topic'] = result['topic'].astype('category')    

Assigning affinity rate, depending on number of events, regardless of source_system

In [22]:
result['affinity'] = (result.events*10/tresh)
result['affinity'] = result['affinity'].clip(upper=10).astype('int8')

In [17]:
# result100 = result.sample(n=100000)

# Saving results

In [24]:
result_path = ''
result_filename = 'affinity'
# result100.to_csv(result_path+result_filename+'.csv', index=False)
result.to_csv(result_path+result_filename+'.csv', index=False)

In [23]:
result.memory_usage(deep=True)

Index                        132
scv_id                   7813956
country                  1953656
topic                    1966297
events_PX                3906978
events_activecampaign    3906978
events_catalogue         3906978
events                   3906978
affinity                 1953489
dtype: int64